<a href="https://colab.research.google.com/github/udlbook/udlbook/blob/main/Notebooks/Chap12/12_1_Self_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 12.1: Self Attention**

This notebook builds a self-attention mechanism from scratch, as discussed in section 12.2 of the book.

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

The self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.

In [2]:
# Set seed so we get the same random numbers
np.random.seed(3)
# Number of inputs
N = 3
# Number of dimensions of each input
D = 4
# Create an empty list
all_x = []
# Create elements x_n and append to list
for n in range(N):
  all_x.append(np.random.normal(size=(D,1)))
# Print out the list
print(all_x)

[array([[ 1.78862847],
       [ 0.43650985],
       [ 0.09649747],
       [-1.8634927 ]]), array([[-0.2773882 ],
       [-0.35475898],
       [-0.08274148],
       [-0.62700068]]), array([[-0.04381817],
       [-0.47721803],
       [-1.31386475],
       [ 0.88462238]])]


We'll also need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4)

In [3]:
# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters
omega_q = np.random.normal(size=(D,D))
omega_k = np.random.normal(size=(D,D))
omega_v = np.random.normal(size=(D,D))
beta_q = np.random.normal(size=(D,1))
beta_k = np.random.normal(size=(D,1))
beta_v = np.random.normal(size=(D,1))

Now let's compute the queries, keys, and values for each input

In [46]:
# Make three lists to store queries, keys, and values
all_queries = []
all_keys = []
all_values = []
# For every input
for x in all_x:
  # TODO -- compute the keys, queries and values.
  # Replace these three lines
  query = np.matmul(omega_q, x) + beta_q
  key = np.matmul(omega_k, x) + beta_k
  value = np.matmul(omega_v, x) + beta_v

  all_queries.append(query)
  all_keys.append(key)
  all_values.append(value)

We'll need a softmax function (equation 12.5) -- here, it will take a list of arbitrary numbers and return a list where the elements are non-negative and sum to one

In [47]:
def softmax(items_in):

  # TODO Compute the elements of items_out
  # Replace this line
  total = np.exp(items_in).sum()
  items_out = np.exp(items_in) / total

  return items_out ;

In [48]:
m1 = np.array([1,2,3,4])
softmax(m1)

array([0.0320586 , 0.08714432, 0.23688282, 0.64391426])

Now compute the self attention values:

In [50]:
all_x_prime = []

for n in range(N):
  all_dots = []
  # compute dot products
  for key in all_keys:
    dot_product = np.dot(key.T, all_queries[n])
    all_dots.append(dot_product)

  # Softmax the dot products
  alphas = softmax(all_dots)

  # TODO: Compute a weighted sum of all of the values according to the attention
  # (equation 12.3)
  # Replace this line

  # Do the weigted sum of all alphas
  sum = np.zeros_like(all_values[0])
  for m in range(N):
    sum += alphas[m]*all_values[m]

  all_x_prime.append(sum)

# Print out true values to check you have it correct
print("x_prime_0_calculated:", all_x_prime[0].transpose())
print("x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]")
print("x_prime_1_calculated:", all_x_prime[1].transpose())
print("x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]")
print("x_prime_2_calculated:", all_x_prime[2].transpose())
print("x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]")

x_prime_0_calculated: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_1_calculated: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_2_calculated: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]
x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]


Now let's compute the same thing, but using matrix calculations.  We'll store the $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ in the columns of a $D\times N$ matrix, using equations 12.6 and 12.7/8.

Note:  The book uses column vectors (for compatibility with the rest of the text), but in the wider literature it is more normal to store the inputs in the rows of a matrix;  in this case, the computation is the same, but all the matrices are transposed and the operations proceed in the reverse order.

In [10]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in) ;
  # Sum over columns
  denom = np.sum(exp_values, axis = 0);
  # Replicate denominator to N rows
  denom = np.matmul(np.ones((data_in.shape[0],1)), denom[np.newaxis,:])
  # Compute softmax
  softmax = exp_values / denom
  # return the answer
  return softmax

In [31]:
# Now let's compute self attention in matrix form
def self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):

  # TODO -- Write this function
  # 1. Compute queries, keys, and values
  queries = np.matmul(omega_q, X) + beta_q
  keys = np.matmul(omega_k, X) + beta_k
  values = np.matmul(omega_v, X) + beta_v
  # 2. Compute dot products
  dots = np.matmul(keys.T, queries)
  # 3. Apply softmax to calculate attentions
  attentions = softmax_cols(dots)
  print(f'Attentions: {attentions}\n')
  # 4. Weight values by attentions
  # Replace this line
  X_prime = np.matmul(values, attentions)

  return X_prime

In [32]:
# Copy data into matrix
X = np.zeros((D, N))
X[:,0] = np.squeeze(all_x[0])
X[:,1] = np.squeeze(all_x[1])
X[:,2] = np.squeeze(all_x[2])

# Run the self attention mechanism
X_prime = self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[1.24326146e-13 2.79525306e-12 5.05707907e-03]
 [9.98281489e-01 5.85506360e-03 6.54776072e-03]
 [1.71851130e-03 9.94144936e-01 9.88395160e-01]]
[[ 0.94744244  1.64201168  1.61949281]
 [-0.24348429 -0.08470004 -0.06641533]
 [-0.91310441  4.02764044  3.96863308]
 [-0.44522983  2.18690791  2.15858316]]


If you did this correctly, the values should be the same as above.

TODO:  

Print out the attention matrix
You will see that the values are quite extreme (one is very close to one and the others are very close to zero.  Now we'll fix this problem by using scaled dot-product attention.

In [37]:
# Now let's compute self attention in matrix form
def scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):

  dim = omega_q.shape[0]
  # TODO -- Write this function
  # 1. Compute queries, keys, and values
  queries = np.matmul(omega_q, X) + beta_q
  keys = np.matmul(omega_k, X) + beta_k
  values = np.matmul(omega_v, X) + beta_v
  # 2. Compute dot products
  dots = np.matmul(keys.T, queries)
  # 3. Scale the dot products as in equation 12.9
  scaled_dots = dots / np.sqrt(dim)
  # 4. Apply softmax to calculate attentions
  attentions = softmax_cols(scaled_dots)
  print(f'Scaled Attentions: {attentions}\n')
  # 5. Weight values by attentions
  # Replace this line
  X_prime = np.matmul(values, attentions)

  return X_prime

In [38]:
# Run the self attention mechanism
X_prime = scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[3.38843552e-07 1.55730194e-06 6.20418746e-02]
 [9.60161968e-01 7.12734969e-02 7.05962187e-02]
 [3.98376935e-02 9.28724946e-01 8.67361907e-01]]
[[ 0.97411966  1.59622051  1.32638014]
 [-0.23738409 -0.09516106  0.13062402]
 [-0.72333202  3.70194096  3.02371664]
 [-0.34413007  2.01339538  1.6902419 ]]


TODO -- Investigate whether the self-attention mechanism is covariant with respect to permutation.
If it is, when we permute the columns of the input matrix $\mathbf{X}$, the columns of the output matrix $\mathbf{X}'$ will also be permuted.

In [45]:
# Try intial X values
original_result = scaled_dot_product_self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Permute the X values
X_prime = X + np.random.rand(*X.shape)

permuted_result = scaled_dot_product_self_attention(X_prime, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

print(f'Original result: {original_result}\nPermuted result: {permuted_result}')

[[3.38843552e-07 1.55730194e-06 6.20418746e-02]
 [9.60161968e-01 7.12734969e-02 7.05962187e-02]
 [3.98376935e-02 9.28724946e-01 8.67361907e-01]]
[[1.21930268e-04 1.23199802e-02 9.99978851e-01]
 [9.99841385e-01 4.74892227e-01 1.10319797e-05]
 [3.66851420e-05 5.12787793e-01 1.01165569e-05]]
Original result: [[ 0.97411966  1.59622051  1.32638014]
 [-0.23738409 -0.09516106  0.13062402]
 [-0.72333202  3.70194096  3.02371664]
 [-0.34413007  2.01339538  1.6902419 ]]
Permuted result: [[-1.26086322 -1.03932582 -4.73638683]
 [ 0.98240379  1.18119186  4.77826934]
 [-2.61861573 -0.47680921 -8.58930963]
 [-1.33690986 -0.31433168 -4.11249652]]


The by-hand version does not include biases. Additionally, the size of the query, key, and value weight matrices are different -- in the book, they are $D \times N$, where $N$ is the number of input embeddings and $D$ is the length of those embeddings. In the by hand example, they are $3 \times N$, and $D \neq 3$. The result of this is that the $\textbf{Q}$, $\textbf{K}$, and $\textbf{V}$ matrices have different dimensions in each example. In the book, the $\textbf{Q}$, $\textbf{K}$, and $\textbf{V}$ matrices have the same dimensions as the inputs ($D \times N$), while in the by hand example, they are $3 \times N$. However, the dot product matrix does have the same dimensions as in the book, because the $N$ dimension is preserved. Otherwise, the process is largely the same.